<a href="https://colab.research.google.com/github/splendidcomputer/cora_project/blob/main/Mostafa_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!git clone https://github.com/splendidcomputer/cora_project/blob/main/cora.content

Cloning into 'cora.content'...
fatal: repository 'https://github.com/splendidcomputer/cora_project/blob/main/cora.content/' not found


In [55]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from keras.utils import np_utils
from tensorflow.keras.callbacks import Callback

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
cora = pd.read_csv("./cora.content", delimiter="\t", header=None)
y_labels = ["Case_Based", "Genetic_Algorithms", "Neural_Networks", "Probabilistic_Methods", "Reinforcement_Learning", "Rule_Learning", "Theory"]

for i,label in enumerate(y_labels):
    cora[1434][cora[1434]==label] = i

<ipython-input-2-3b73d10176e0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cora[1434][cora[1434]==label] = i


In [ ]:
X = np.asarray(cora.iloc[:,1:-1]).astype('float32')
y = np.asarray(cora.iloc[:,-1]).astype('float32')

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)


In [ ]:
model = Sequential()
model.add(layers.Dense(512, input_shape=(1433,), activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))
# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
# Implement callback function to stop training
# when accuracy reaches e.g. ACCURACY_THRESHOLD = 0.95

VALIDATION_ACCURACY_THRESHOLD = 0.75

class myCallback(Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_accuracy') > VALIDATION_ACCURACY_THRESHOLD):   
          print("\nReached %2.2f%% accuracy, so stopping training!!" %(VALIDATION_ACCURACY_THRESHOLD*100))   
          self.model.stop_training = True

callbacks = myCallback()


In [ ]:
kFold = StratifiedKFold(n_splits=10)
# fit the keras model on the dataset
for train, test in kFold.split(X, encoded_Y):
  # convert integers to dummy variables (i.e. one hot encoded)
  dummy_y = np_utils.to_categorical(encoded_Y)
  model.fit(X[train], dummy_y[train], validation_data=(X[test], dummy_y[test]), epochs=20, batch_size=1, callbacks=[callbacks])

Epoch 1/20
2437/2437 [==============================] - 10s 3ms/step - loss: 1.3993 - accuracy: 0.4908 - val_loss: 1.0211 - val_accuracy: 0.6199
Epoch 2/20
2437/2437 [==============================] - 7s 3ms/step - loss: 0.8294 - accuracy: 0.7144 - val_loss: 1.0697 - val_accuracy: 0.6642
Epoch 3/20
2437/2437 [==============================] - 7s 3ms/step - loss: 0.4411 - accuracy: 0.8580 - val_loss: 1.1156 - val_accuracy: 0.7417
Epoch 4/20
2434/2437 [============================>.] - ETA: 0s - loss: 0.2373 - accuracy: 0.9322
Reached 75.00% accuracy, so stopping training!!
2437/2437 [==============================] - 7s 3ms/step - loss: 0.2371 - accuracy: 0.9323 - val_loss: 1.5078 - val_accuracy: 0.7675
Epoch 1/20
2434/2437 [============================>.] - ETA: 0s - loss: 0.3179 - accuracy: 0.9339
Reached 75.00% accuracy, so stopping training!!
2437/2437 [==============================] - 7s 3ms/step - loss: 0.3188 - accuracy: 0.9335 - val_loss: 0.1261 - val_accuracy: 0.9631
Epoch 1/2

In [ ]:
list_predictions = []
folds_val_acc = []
for train, test in kFold.split(X, encoded_Y):
  fold_predictions = np.argmax(model.predict(X[test]), axis=1)
  list_predictions.append(fold_predictions) 
  dummy_y = np_utils.to_categorical(encoded_Y)
  folds_val_acc.append(model.evaluate(X[test], dummy_y[test]))

9/9 [==============================] - 0s 3ms/step - loss: 0.0039 - accuracy: 1.0000


In [ ]:
folds_val_acc

[[0.014322157017886639, 0.9963099360466003],
 [0.0005648264777846634, 1.0],
 [0.02704770304262638, 0.9926199316978455],
 [0.012001010589301586, 0.9926199316978455],
 [0.0006334340432658792, 1.0],
 [4.1473711462458596e-05, 1.0],
 [0.001961861504241824, 1.0],
 [0.0276322141289711, 0.9926199316978455],
 [0.010744171217083931, 0.9925925731658936],
 [0.003907768987119198, 1.0]]

In [ ]:
list_predictions

[array([2, 5, 4, 4, 3, 3, 6, 2, 2, 6, 2, 1, 3, 0, 2, 2, 4, 2, 2, 2, 6, 4,
        1, 3, 4, 6, 2, 3, 2, 5, 0, 2, 0, 2, 4, 4, 2, 4, 1, 4, 6, 3, 1, 2,
        0, 4, 2, 0, 3, 0, 0, 3, 6, 1, 6, 1, 6, 4, 2, 2, 4, 0, 3, 4, 0, 1,
        0, 3, 2, 0, 6, 3, 2, 4, 3, 0, 3, 2, 6, 6, 2, 3, 4, 2, 3, 4, 5, 5,
        6, 6, 2, 2, 4, 3, 3, 3, 0, 2, 6, 3, 1, 2, 2, 4, 2, 3, 5, 3, 3, 6,
        2, 2, 2, 2, 6, 3, 0, 3, 4, 5, 1, 2, 2, 2, 2, 2, 3, 2, 4, 1, 6, 3,
        4, 2, 2, 5, 5, 3, 3, 1, 6, 1, 2, 2, 2, 1, 2, 5, 4, 0, 0, 6, 2, 2,
        2, 3, 5, 6, 5, 6, 2, 3, 1, 6, 5, 0, 1, 2, 3, 6, 0, 2, 2, 5, 6, 5,
        2, 2, 6, 3, 0, 5, 3, 2, 1, 1, 2, 1, 2, 2, 2, 5, 5, 2, 6, 2, 2, 2,
        2, 2, 2, 2, 2, 6, 1, 3, 2, 2, 0, 5, 2, 3, 1, 0, 0, 5, 0, 2, 6, 1,
        2, 2, 0, 2, 2, 2, 2, 0, 1, 2, 1, 2, 6, 1, 2, 0, 3, 3, 3, 6, 2, 6,
        0, 0, 3, 0, 6, 1, 6, 3, 1, 3, 3, 3, 6, 1, 6, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]),
 array([4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 5, 4, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       

In [ ]:
final_predictions = []
for fold_prediction in list_predictions:
  for item in fold_prediction:
    final_predictions.append(y_labels[item])

In [ ]:
final_predictions_array = np.array([cora.iloc[:,0],final_predictions]).T

In [ ]:
final_predictions_array

array([['31336', 'Neural_Networks'],
       ['1061127', 'Rule_Learning'],
       ['1106406', 'Reinforcement_Learning'],
       ...,
       ['1128978', 'Genetic_Algorithms'],
       ['117328', 'Case_Based'],
       ['24043', 'Neural_Networks']], dtype='<U22')

In [ ]:
list_ids = list(cora.iloc[:,0])

In [ ]:
d = {'IDs': list_ids, 'Predictions': final_predictions}
df = pd.DataFrame(data=d)

In [ ]:
df.to_csv('final_predictions.tsv', sep="\t")